Import useful libraires

In [5]:
import pandas as pd
import numpy as np
import requests
import csv
import re
import bs4
import time
from bs4 import BeautifulSoup
import glob
import os

Get the informations about all the clubs in Switzerland

In [2]:
url_clubs='http://www.residentadvisor.net/clubs.aspx?ai=60'
r=requests.get(url_clubs)
soup=BeautifulSoup(r.content,'html.parser')

Get the indexes that will help searching the soup to get the clubs info

In [3]:
list_of_indexes=[]
for indexes in soup.findAll('div',{'class':'pb8 pr8 clearfix'}):
    list_=indexes.findAll('a')
    for index_ in list_:
        index=index_.contents
        list_of_indexes.append(index)

We now have a list of all indexes to get through the soup. The last one 'closed' will be deleted

In [4]:
list_of_indexes

[['A'],
 ['B'],
 ['C'],
 ['D'],
 ['E'],
 ['F'],
 ['G'],
 ['H'],
 ['I'],
 ['J'],
 ['K'],
 ['L'],
 ['M'],
 ['N'],
 ['O'],
 ['P'],
 ['Q'],
 ['R'],
 ['S'],
 ['T'],
 ['U'],
 ['V'],
 ['W'],
 ['X'],
 ['Y'],
 ['Z'],
 ['0-9'],
 ['Closed']]

In [5]:
del list_of_indexes[-1]

Get the clubs information using the indexes

In [6]:
list_of_clubs=pd.DataFrame(columns=['name','ref','address'])

for index in list_of_indexes:
    try:
        list_raw=soup.find('li',{'id':index})
        for club_raw in list_raw.findAll('li',{'class':'clearfix'}):
            club_ref=club_raw.find('a').get('href')
            if len(club_raw.find('a'))>0:
                club_name=club_raw.find('a').contents[0]
            if len(club_raw.find('div', {'class':'fl grey mobile-off'}))>0:
                club_adress=club_raw.find('div', {'class':'fl grey mobile-off'}).contents[0]
            club={}
            club['name']=club_name
            club['ref']=club_ref
            club['address']=club_adress
            list_of_clubs=list_of_clubs.append(club, ignore_index=True)
    except:
        print('Error while parsing for club:'+str(club_name)+'for index '+str(index))
        import traceback;
        traceback.print_exc()
        time.sleep(20)

In [7]:
list_of_clubs

,name,ref,address
0,Abraxas,/club.aspx?id=31409,"Chemin du Stand 5; Pully, 1009"
1,Acanto,/club.aspx?id=39607,"Pfingstweidstrasse 6, 8005 Zürich"
2,Acapella Bar,/club.aspx?id=102059,"Place de la Gare, 1957 Ardon, Valais, Suisse"
3,Acqua Lounge Basilea,/club.aspx?id=18481,"Binningerstrasse 14; 4051, Basel"
4,Adagio,/club.aspx?id=9913,"Seidenhofstrasse 5; 6002, Luzern"
5,Akt,/club.aspx?id=74860,"Niederdorfstrasse 64, 8001 Zürich"
6,Albani Music Club,/club.aspx?id=25492,Steinberggasse 16
7,Alhambar,/club.aspx?id=24542,"Rue de la Rôtisserie 10; 1204, Genève"
8,Alice Choo,/club.aspx?id=102769,"Limmatstrasse 275, 8005 Zürich"
9,Alpina Hütte,/club.aspx?id=114959,"Bergstation Corviglia, 7500 St. Moritz"


Add a column with the identificateur of the club

In [8]:
def get_number (name):
    out=re.search(r'\d+', name).group()
    return(out)

In [9]:
list_of_clubs['identificateur']=list_of_clubs['ref'].apply(get_number)

In [10]:
FolderName='ResidentAdvisor'

Now getting the events

In [ ]:
url_events='https://www.residentadvisor.net'
for index, rows in list_of_clubs.iterrows():
    try:
        list_of_events_=pd.DataFrame(columns=['club_adress','club_id','club_name','club_ref','date','ref_event'])
        for year in range (2006,2018):
            url=url_events+rows['ref']+'&show=events'
            if year<2017:
                url=url+'&yr='+str(year)    
            r=requests.get(url)
            soup=BeautifulSoup(r.content,'html.parser')
            list_of_events=soup.find('div',{'id':'divArchiveEvents'})
            for event_ in list_of_events.findAll('li',{'style':'min-height:220px;'}):
                url_2=url_events+event_.find('a').get('href')
                r=requests.get(url_2)
                soup=BeautifulSoup(r.content,'html.parser')
                line_up=soup.find('p',{'class':'lineup large'})
            
                if line_up is None:
                    line_up=soup.find('p',{'class':'lineup medium'})
                if line_up is not None:            
                    for artist in line_up.findAll('a'):
                        event={}
                        event['artiste']=artist.contents[0]
                        event['club_adress']=rows['address']
                        event['club_id']=rows['identificateur']
                        event['club_name']=rows['name']
                        event['club_ref']=rows['ref']
                        event['date']=event_.find('p',{'class':'date'}).contents[0]
                        event['ref_event']=event_.find('a').get('href')
                        list_of_events_=list_of_events_.append(event, ignore_index=True)
    
        list_of_events_=list_of_events_.drop_duplicates()
        name=rows['name']
        name = name.replace("/", "")
        csv_=FolderName+'/'+rows['name']+'.csv'
        if len(list_of_events_)>0:
            pd.DataFrame(list_of_events_,columns=list(list_of_events_.columns)).to_csv(csv_, index=False, encoding='utf-8')
            stop
    except:
        print(rows['name'])
        
        time.sleep(20)


        
    

#### One csv to rule them all

In [16]:
#Fetch all csvs and concat them in a DataFrame
path =r'ResidentAdvisor/' # use your path
allFiles = glob.glob(os.path.join(path, "*.csv"))
df = pd.concat((pd.read_csv(f) for f in allFiles), ignore_index=True).drop(['club_ref', 'club_id', 'ref_event'], 1)

In [17]:
#Write the DataFrame to a csv file

filename = 'total_residentadvisor.csv'
pd.DataFrame(df, columns=list(df.columns)).to_csv(filename, index=False, encoding="utf-8")
print('ResidentAdvisor data saved to file')

ResidentAdvisor data saved to file
